# Local RAG pipeline

In [18]:
import torch
from datetime import datetime

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [20]:
def showTime():
    return str("["+datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')+" UTC]")

In [21]:
# download pdf
import os
import requests
pdf_path = "human-nutrition-text.pdf"

if not os.path.exists(pdf_path):
    print(f"{showTime()}[INFO] File does not exist, downloading...")
    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"
    filename = pdf_path

    # send request
    response = requests.get(url)

    if response.status_code == 200:
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"{showTime()}[INFO] File downloaded and saved as {filename}")
    else:
        print(f"{showTime()}[INFO] Failed to download. {response.status_code}")
else:
    print(f"{showTime()}[INFO] File exists.")    

[2024-10-04 20:15:52.988538 UTC][INFO] File does not exist, downloading...
[2024-10-04 20:16:18.601700 UTC][INFO] File downloaded and saved as human-nutrition-text.pdf


In [31]:
# open pdf
import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    """Performs minor forating on text"""
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({"page_number": page_number-41, # page number where real book content starts
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text)/4, # 1 token = 4 characters
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:3]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''},
 {'page_number': -39,
  'page_char_count': 320,
  'page_word_count': 54,
  'page_sentence_count_raw': 1,
  'page_token_count': 80.0,
  'text': 'Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  KAINOA FIALKOWSKI REVILLA,  JENNIFER DRAPER, GEMADY  LANGFELDER, CHERYL GIBBY, CHYNA  NICOLE CHUN, AND ALLISON  CALABRESE'}]

In [32]:
import random
random.sample(pages_and_texts, k=3)

[{'page_number': 472,
  'page_char_count': 641,
  'page_word_count': 105,
  'page_sentence_count_raw': 5,
  'page_token_count': 160.25,
  'text': 'Photo by  Hope House  Press on  unsplash.co m / CC0  https://unspl ash.com/ photos/ PJzc7LOt2Ig  Weight Management  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  “Obesogenic” is a word that has sprung up in the language of public  health professionals in the last two decades. The Centers for  Disease Control and Prevention (CDC) defines obesogenic as “an  environment that promotes increased food intake, non-healthful  foods, and physical inactivity.”1  1. Obesogenic Environments. Center for Disease Control  and Prevention (CDC). https://www.cdc.gov/pcd/ 472  |  Weight Management'},
 {'page_number': 780,
  'page_char_count': 1155,
  'page_word_count': 202,
  'page_sentence_count_raw': 12,
  'page_token_count': 288.75,
  'text': 'Learning Objectives  By the end of this chapter you will be

In [33]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,147,3,199.25,Contents Preface University of Hawai‘i at Mā...


In [34]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,199.50,10.52,287.00
std,348.86,560.38,95.83,6.55,140.10
min,-41.00,0.00,1.00,1.00,0.00
25%,260.75,762.00,134.00,5.00,190.50
50%,562.50,1231.50,216.00,10.00,307.88
75%,864.25,1603.50,272.00,15.00,400.88
max,1166.00,2308.00,430.00,39.00,577.00


In [41]:
from spacy.lang.en import English

nlp = English()
# https://spacy.io/api/sentencizer
nlp.add_pipe("sentencizer")

# create example document instance
doc = nlp("This is sentence. This id second sentence. I like trains.")
assert len(list(doc.sents)) == 3

list(doc.sents)


[This is sentence., This id second sentence., I like trains.]

In [43]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)
    
    # make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    # count sentences
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [47]:
random.sample(pages_and_texts, k=1)

[{'page_number': 873,
  'page_char_count': 1677,
  'page_word_count': 291,
  'page_sentence_count_raw': 17,
  'page_token_count': 419.25,
  'text': 'children should be provided nutrient-dense food at meal- and  snack-time. However, it is important not to overfeed children, as  this can lead to childhood obesity, which is discussed in the next  section. Parents and other caregivers can turn to the MyPlate  website for guidance: http://www.choosemyplate.gov/.  Macronutrients  For carbohydrates, the Acceptable Macronutrient Distribution  Range (AMDR) is 45–65 percent of daily calories (which is a  recommended daily allowance of 135–195 grams for 1,200 daily  calories). Carbohydrates high in fiber should make up the bulk of  intake. The AMDR for protein is 10–30 percent of daily calories  (30–90 grams for 1,200 daily calories). Children have a high need for  protein to support muscle growth and development. High levels of  essential fatty acids are needed to support growth (although not as

In [50]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)


,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,199.50,10.52,287.00,10.32
std,348.86,560.38,95.83,6.55,140.10,6.30
min,-41.00,0.00,1.00,1.00,0.00,0.00
25%,260.75,762.00,134.00,5.00,190.50,5.00
50%,562.50,1231.50,216.00,10.00,307.88,10.00
75%,864.25,1603.50,272.00,15.00,400.88,15.00
max,1166.00,2308.00,430.00,39.00,577.00,28.00


In [52]:
# chunk sentences together

num_sentence_chunk_size = 10

def split_list(input_list: list[str], slice_size: int=num_sentence_chunk_size) -> list[list[str]]:
    return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]

test_list = list(range(25))
split_list(test_list)


[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [53]:
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"], slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [54]:
random.sample(pages_and_texts, k=1)

[{'page_number': 769,
  'page_char_count': 1918,
  'page_word_count': 309,
  'page_sentence_count_raw': 12,
  'page_token_count': 479.5,
  'text': 'collaboration with FAO, continually reviews new research and  information  from  around  the  world  on  human  nutrient  requirements and recommended nutrient intakes. This is a vast  and never-ending task, given the large number of essential human  nutrients. These nutrients include protein, energy, carbohydrates,  fats and lipids, a range of vitamins, and a host of minerals and trace  elements.  Many countries rely on WHO and FAO to establish and  disseminate this information, which they adopt as part of their  national dietary allowances. Others use it as a base for their  standards. The establishment of human nutrient requirements is the  common foundation for all countries to develop food-based dietary  guidelines for their populations.  Establishing requirements means that the public health and  clinical significance of intake levels

In [55]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,199.50,10.52,287.00,10.32,1.53
std,348.86,560.38,95.83,6.55,140.10,6.30,0.64
min,-41.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,260.75,762.00,134.00,5.00,190.50,5.00,1.00
50%,562.50,1231.50,216.00,10.00,307.88,10.00,1.00
75%,864.25,1603.50,272.00,15.00,400.88,15.00,2.00
max,1166.00,2308.00,430.00,39.00,577.00,28.00,3.00


In [58]:
# spliting each chunk into each own item

import re

pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        # join list of sentences into paragraph
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        # add " " before start of sentece and after .
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk)/4

        pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)
        
        

  0%|          | 0/1208 [00:00<?, ?it/s]

1843

In [60]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 522,
  'sentence_chunk': 'are packaged into the lipid-containing chylomicrons inside small intestine mucosal cells and then transported to the liver. In the liver, carotenoids are repackaged into lipoproteins, which transport them to cells. The retinoids are aptly named as their most notable function is in the retina of the eye where they aid in vision, particularly in seeing under low-light conditions. This is why night blindness is the most definitive sign of vitamin A deficiency. Vitamin A has several important functions in the body, including maintaining vision and a healthy immune system. Many of vitamin A’s functions in the body are similar to the functions of hormones (for example, vitamin A can interact with DNA, causing a change in protein function). Vitamin A assists in maintaining healthy skin and the linings and coverings of tissues; it also regulates growth and development. As an antioxidant, vitamin A protects cellular membranes, helps in maintaining glut

In [61]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,583.38,734.10,112.74,183.52
std,347.79,447.51,71.24,111.88
min,-41.00,12.00,3.00,3.00
25%,280.50,315.00,45.00,78.75
50%,586.00,745.00,115.00,186.25
75%,890.00,1118.00,173.00,279.50
max,1166.00,1830.00,297.00,457.50


In [62]:
# filter chunks of text for short chunks
min_token_length = 30

for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f"Chunk token count: {row[1]['chunk_token_count']} | Text: {row[1]['sentence_chunk']}")

Chunk token count: 16.5 | Text: Table 4.6 Sweeteners Carbohydrates and Personal Diet Choices | 281
Chunk token count: 9.0 | Text: 1088 | Nutrition, Health and Disease
Chunk token count: 26.25 | Text: Snowdon W, Osborn T. (2003). Coconut: It’s role in health. Secretariat of the Pacific. 292 | Introduction
Chunk token count: 25.25 | Text: PART XV CHAPTER 15. LIFESPAN NUTRITION IN ADULTHOOD Chapter 15. Lifespan Nutrition in Adulthood | 901
Chunk token count: 6.5 | Text: Fat-Soluble Vitamins | 537


In [75]:
# filter dataframe for rows with under 30 tokens
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52.5}]

In [67]:
random.sample(ages_and_chunks_over_min_token_len, k=2)

[{'page_number': 480,
  'sentence_chunk': 'The sum of caloric expenditure is referred to as total energy expenditure (TEE). Basal metabolism refers to those metabolic pathways necessary to support and maintain the body’s basic functions (e.g. breathing, heartbeat, liver and kidney function) while at rest. The basal metabolic rate (BMR) is the amount of energy required by the body to conduct its basic functions over a certain time period. The great majority of energy expended (between 50 and 70 percent) daily is from conducting life’s basic processes. Of all the organs, the liver requires the most energy 480 | Weight Management',
  'chunk_char_count': 591,
  'chunk_word_count': 95,
  'chunk_token_count': 147.75},
 {'page_number': 1006,
  'sentence_chunk': 'Amanita Muscaria by Onder Wijsgek / CC BY 3.0 Poisonous Mushrooms Like molds, mushrooms are fungi and the poisonous kind produces mycotoxins that can cause food intoxication. Toxic mushrooms, also known as toadstools, can cause severe

In [69]:
# embeding text chunks
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device=device)

sentences = ["Sentence Transformers  is the go-to Python module for accessing, using, and training state-of-the-art text and image embedding models.",
             "It can be used to compute embeddings using Sentence Transformer models.",
             "Or to calculate similarity scores using Cross-Encoder models (quickstart).",
             "I like oranges!"
            ]

embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

for sentence, embedding in embeddings_dict.items():
    print(f"Sentence: {sentence} | Embedding: {embedding}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

F:\OneDrive\github\simple-local-rag\venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in F:\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Sentence: Sentence Transformers  is the go-to Python module for accessing, using, and training state-of-the-art text and image embedding models. | Embedding: [ 5.34216687e-02  2.65813842e-02 -4.48918482e-03  5.93200512e-02
 -2.61534285e-02  1.94523521e-02  5.60242543e-03 -2.80467956e-03
  8.21539573e-03 -8.18702057e-02  1.94471120e-03  6.86734496e-03
 -2.91680321e-02  4.43347171e-02  6.03051204e-03 -1.03364438e-01
  7.97539111e-03  7.63178628e-04  1.06992887e-03  3.44491303e-02
  3.61879170e-02  1.65099427e-02  1.89586747e-02  7.07316101e-02
 -4.80418317e-02 -1.90921985e-02  2.15894151e-02  2.40902938e-02
  4.91196616e-03 -7.31106009e-03  1.90134346e-02  1.89336725e-02
  5.87720945e-02  7.21790036e-03  1.62987783e-06  4.75013442e-02
  1.77877303e-02 -1.53129799e-02  2.65215486e-02  9.36527364e-03
  9.98330638e-02 -7.21652582e-02  1.59420054e-02  1.77639723e-02
 -3.04701831e-02  1.50883812e-02  4.88863066e-02  8.50658417e-02
  7.33739659e-02  8.73203278e-02 -2.37938538e-02 -2.61304732e-

In [70]:
embeddings[0].shape

(768,)

In [73]:
embedding = embedding_model.encode("I like trains.")
embedding

array([-6.61777407e-02,  1.08034439e-01, -1.73398517e-02,  2.59081796e-02,
        1.68851223e-02,  3.35716531e-02, -2.45636888e-02, -1.88797992e-02,
       -4.09865156e-02, -1.09141804e-02,  5.84466010e-03,  4.02872497e-03,
       -6.20981157e-02, -4.92319465e-02,  2.45086267e-03, -6.47033751e-02,
        3.94016244e-02, -2.31145285e-02, -1.03348777e-01,  3.15237306e-02,
        2.37877923e-03,  4.83119935e-02, -4.25290577e-02, -1.39789656e-02,
       -2.20304728e-02,  3.79067753e-03, -2.42629256e-02, -6.35410175e-02,
        3.40222046e-02,  4.36703116e-02, -1.78661961e-02,  5.34686213e-03,
        3.54878651e-03,  3.20020095e-02,  1.59164517e-06,  4.62644622e-02,
       -3.53364311e-02,  1.27375694e-02,  2.53395680e-02, -2.91919559e-02,
        4.92486767e-02, -6.11238088e-03, -9.67751537e-03, -2.03889981e-02,
       -2.21629092e-03, -1.74699593e-02,  5.17543368e-02,  3.11498214e-02,
       -8.40535574e-03,  3.38792019e-02, -1.15190363e-02, -2.02138815e-02,
       -1.07733719e-01, -

In [77]:
%%time
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/1680 [00:00<?, ?it/s]

CPU times: total: 2min 49s
Wall time: 33.6 s


In [78]:
%%time
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]
text_chunks[100]

CPU times: total: 0 ns
Wall time: 1 ms


'Nutrition Assessment Nutritional assessment is the interpretation of anthropometric, biochemical (laboratory), clinical and dietary data to determine whether a person or groups of people are well nourished or malnourished (overnourished or undernourished). Nutritional assessment can be done using the ABCD methods. These refer to the following: • A. Anthropometry • B. Biochemical methods • C. Clinical methods • D. Dietary methods Anthropometry methods of assessing nutritional status The word anthropometry comes from two words: Anthropo means ‘human’ and metry means ‘measurement’. The different measurements taken to assess growth and body composition are presented below. To assess growth, several different measurements including length, height, weight, head circumference, mid-arm circumference, skin-fold thickness, head/chest ratio, and hip/waist ratio can be used. Height and weight measurements are essential in children to evaluate physical growth. As an additional resource, the NHANES

In [80]:
%%time
# embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks, batch_size=16, convert_to_tensor=True)
text_chunk_embeddings[100]

CPU times: total: 1min 17s
Wall time: 17 s


tensor([ 1.2773e-02, -1.0473e-02,  7.8695e-03, -3.4675e-03,  9.6361e-05,
        -3.3710e-03,  1.4906e-02,  1.1292e-02,  3.0318e-02, -9.5284e-03,
         8.3516e-02, -1.0757e-01,  6.2119e-02, -8.9779e-03, -9.7639e-03,
        -1.9894e-03, -1.8487e-02, -1.7352e-02, -2.0593e-02,  6.3473e-02,
        -6.6939e-02, -4.7820e-03, -9.5209e-03, -1.8517e-02, -4.3938e-02,
         4.6283e-02,  3.2606e-02, -2.0607e-02, -1.8851e-02, -5.1422e-02,
         1.2133e-02, -9.4620e-03, -1.3889e-02, -3.8846e-02,  1.9150e-06,
        -4.6082e-02,  1.8463e-02,  2.5674e-02, -1.9828e-02,  2.2107e-02,
         7.4989e-02, -5.6563e-02, -1.7632e-02, -3.8679e-02,  7.9016e-03,
        -8.7244e-03,  5.5975e-02,  3.4743e-04, -3.4352e-02, -1.5650e-02,
         2.4092e-02, -5.0074e-02, -3.4213e-02, -1.2408e-02, -1.0899e-02,
         3.7432e-02, -2.4751e-03,  7.9598e-02,  5.9286e-02,  3.9348e-02,
        -1.1415e-02,  4.3283e-02,  3.8863e-02,  2.4163e-02, -1.1961e-02,
        -2.1793e-03, -6.3283e-03, -4.6967e-02,  5.4

In [82]:
# save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [84]:
# import saved embeddings
text_chunks_and_embeddings_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embeddings_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,[ 6.74242899e-02 9.02281404e-02 -5.09547861e-...
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,[ 5.52156419e-02 5.92139252e-02 -1.66167393e-...
2,-37,Contents Preface University of Hawai‘i at Māno...,766,116,191.50,[ 2.79801823e-02 3.39813717e-02 -2.06426494e-...
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,144,235.25,[ 6.82566985e-02 3.81274670e-02 -8.46854597e-...
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,[ 3.30264494e-02 -8.49768054e-03 9.57159698e-...
